In [1]:
#import Pkg; Pkg.add("DSP")
#import Pkg; Pkg.add("BenchmarkTools")

In [2]:
using DSP
using BenchmarkTools

In [16]:
B = zeros(7,7);
k = ones(3,3)
B[6, 4] = 5

5

In [17]:
u1 = [1.,1.,1.]
v1 = [1.,1.,1.];
u1 * v1'

3×3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [5]:
function conv1(A, kernel)
    number_of_rows = size(A)[1] - size(kernel)[1] + 1
    number_of_cols = size(A)[2] - size(kernel)[2] + 1
    new_arr = Array{Float64, 2}(undef, number_of_rows, number_of_cols)
    count = 0
    for i in 1:number_of_rows
        for j in 1:number_of_cols
                    i_range = i:i + size(kernel)[1] - 1
                    j_range = j:j + size(kernel)[2] - 1
                    new_arr[i, j] = sum(kernel .* A[i_range, j_range])
        end
    end
    return new_arr
end

conv1 (generic function with 1 method)

In [6]:
function conv2(u, v, A)
    result = conv(u, v, A)
    return result[size(u, 1):end-size(u, 1)+1, size(v, 1):end-size(v, 1)+1]
end

conv2 (generic function with 1 method)

In [7]:
@benchmark conv1($B, $k)

BenchmarkTools.Trial: 
  memory estimate:  311.56 MiB
  allocs estimate:  1992010
  --------------
  minimum time:     215.354 ms (3.68% GC)
  median time:      227.396 ms (3.71% GC)
  mean time:        252.183 ms (12.57% GC)
  maximum time:     394.668 ms (40.28% GC)
  --------------
  samples:          20
  evals/sample:     1

In [8]:
@benchmark conv2($u1, $v1, $B)

BenchmarkTools.Trial: 
  memory estimate:  99.62 MiB
  allocs estimate:  251
  --------------
  minimum time:     274.830 ms (2.68% GC)
  median time:      340.596 ms (21.37% GC)
  mean time:        326.202 ms (15.08% GC)
  maximum time:     377.176 ms (23.15% GC)
  --------------
  samples:          16
  evals/sample:     1

In [14]:
@benchmark fastconv($B, $k)

BenchmarkTools.Trial: 
  memory estimate:  7.66 MiB
  allocs estimate:  9
  --------------
  minimum time:     15.777 ms (0.00% GC)
  median time:      16.209 ms (0.00% GC)
  mean time:        18.141 ms (6.86% GC)
  maximum time:     97.430 ms (83.52% GC)
  --------------
  samples:          276
  evals/sample:     1

In [18]:
res1 = conv1(B, k)
res2 = conv2(u1, v1, B)
res3 = fastconv(B, k);

In [19]:
res1

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  5.0  5.0  5.0  0.0
 0.0  5.0  5.0  5.0  0.0

In [21]:
round.(res2)

5×5 Array{Float64,2}:
 -0.0   0.0  0.0   0.0  -0.0
 -0.0  -0.0  0.0  -0.0  -0.0
  0.0   0.0  0.0   0.0   0.0
  0.0   5.0  5.0   5.0   0.0
 -0.0   5.0  5.0   5.0  -0.0

In [24]:
res3[size(u1, 1):end-size(u1, 1)+1, size(v1, 1):end-size(v1, 1)+1]

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  5.0  5.0  5.0  0.0
 0.0  5.0  5.0  5.0  0.0

In [134]:
u = [1.,1.,1.]
v = [1.,1.,1.];
u * v'

3×3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [135]:
number_of_rows = size(random)[1] - size(kernel)[1] + 1
number_of_cols = size(random)[2] - size(kernel)[2] + 1
m = length(u)+size(random,1)-1
n = length(v)+size(random,2)-1
result = conv(u, v, random)
round.(result)

9×9 Array{Float64,2}:
  0.0   0.0   0.0  -0.0   0.0  -0.0   0.0   0.0   0.0
  0.0  -0.0  -0.0  -0.0  -0.0  -0.0   0.0   0.0   0.0
  0.0  -0.0  -0.0   0.0   0.0   0.0  -0.0  -0.0   0.0
  0.0  -0.0  -0.0  -0.0   0.0  -0.0  -0.0   0.0   0.0
 -0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -0.0   0.0   5.0   5.0   5.0   0.0   0.0   0.0
  0.0  -0.0  -0.0   5.0   5.0   5.0  -0.0   0.0   0.0
  0.0   0.0   0.0   5.0   5.0   5.0   0.0  -0.0  -0.0
  0.0   0.0   0.0   0.0  -0.0  -0.0   0.0   0.0   0.0

In [136]:
round.(result[size(kernel)[1]:end-size(kernel)[1]+1, size(kernel)[2]:end-size(kernel)[2]+1])

5×5 Array{Float64,2}:
 -0.0   0.0  0.0   0.0  -0.0
 -0.0  -0.0  0.0  -0.0  -0.0
  0.0   0.0  0.0   0.0   0.0
  0.0   5.0  5.0   5.0   0.0
 -0.0   5.0  5.0   5.0  -0.0

In [137]:
new_arr

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  5.0  5.0  5.0  0.0
 0.0  5.0  5.0  5.0  0.0

In [83]:
T = random
m = length(u)+size(random,1)-1
n = length(v)+size(random,2)-1
B = zeros(Float64, m, n)

5×9 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [85]:
B[1:size(random,1),1:size(random,2)] = random
B

5×9 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [93]:
fft([u;zeros(Float64, m-length(u))])

5-element Array{Complex{Float64},1}:
                 2.0 + 0.0im               
  1.3090169943749475 - 0.9510565162951535im
 0.19098300562505255 - 0.5877852522924731im
 0.19098300562505255 + 0.5877852522924731im
  1.3090169943749475 + 0.9510565162951535im

In [94]:
fft([v;zeros(Float64,n-length(v))])

9-element Array{Complex{Float64},1}:
                 3.0 + 0.0im               
  1.9396926207859084 - 1.6275953626987474im
 0.23395555688102193 - 1.3268278963378768im
                 0.0 + 0.0im               
  0.8263518223330695 + 0.3007674663608707im
  0.8263518223330695 - 0.3007674663608707im
                 0.0 + 0.0im               
 0.23395555688102193 + 1.3268278963378768im
  1.9396926207859084 + 1.6275953626987474im

In [13]:
using Base.Cartesian
# direct version (do not check if threshold is satisfied)
@generated function fastconv(E::Array{T,N}, k::Array{T,N}) where {T,N}
    quote

        retsize = [size(E)...] + [size(k)...] .- 1
        retsize = tuple(retsize...)
        ret = zeros(T, retsize)

        convn!(ret,E,k)
        return ret

    end
end


# in place helper operation to speedup memory allocations
@generated function convn!(out::Array{T}, E::Array{T,N}, k::Array{T,N}) where {T,N}
    quote
        @inbounds begin
            @nloops $N x E begin
                @nloops $N i k begin
                    (@nref $N out d->(x_d + i_d - 1)) += (@nref $N E x) * (@nref $N k i)
                end
            end
        end
        return out
    end
end

convn! (generic function with 1 method)